In [1]:
# mac475의 ipython 표준 style을 적용함
from IPython.core.display import HTML
styles = open("../styles/custom.css", "r").read()
HTML( styles )

#0. Mode 설정

In [2]:
mode = True
# mode = False

predict_counter = 300

# bracket_pricing별로 구분한다
# bracket = 'Yes'
# bracket = 'No'
bracket = 'All'

#0. Dataset 읽기

In [3]:
import pandas as pd
import numpy as np

df_train = pd.read_csv( './dataset/train_set.csv' )    # data를 읽어들인다.
df_test = pd.read_csv( './dataset/test_set.csv' )    # data를 읽어들인다.

In [4]:
if bracket == 'Yes' :
    # bracket_pricing == Yes인 경우만
    df_train = df_train[ df_train[ 'bracket_pricing' ] == 'Yes' ]
elif bracket == 'No' :
    # bracket_pricing == No인 경우만
    df_train = df_train[ df_train[ 'bracket_pricing' ] == 'No' ]

#1. Feature 추출 function
* quote_date 활용하여 year, month, day 확보

In [5]:
import datetime

def extract_year_month_day_from_quote_date( p_df ) :    # quote date로부터 연도, 월, 날짜를 추출
    p_df[ 'quote_date' ] = pd.to_datetime( p_df[ 'quote_date' ] )    # string을 datetime으로 형변환
    p_df[ 'year' ] = p_df[ 'quote_date' ].dt.year    # 연도
    p_df[ 'month' ] = p_df[ 'quote_date' ].dt.month    # 월
    p_df[ 'day' ] = p_df[ 'quote_date' ].dt.day    # 일
    
#     p_df[ 'weekofyear' ] = p_df[ 'quote_date' ].dt.weekofyear    # 주차
#     p_df[ 'dayofyear' ] = p_df[ 'quote_date' ].dt.dayofyear    # 연중 일자
#     p_df[ 'quarter' ] = p_df[ 'quote_date' ].dt.quarter    # 분기
#     p_df[ 'weekday' ] = p_df[ 'quote_date' ].dt.weekday    # 요일
    
    return p_df

In [6]:
def make_string_date( p_df ) :    # 일자정보를 string으로 변경
    y = p_df[ 'year' ]
    m = p_df[ 'month' ]
    d = p_df[ 'day' ]
    
    str_y = ''
    str_m = ''
    str_d = ''
    
    str_y = str( y )
    str_m = ''
    if m < 10 :
        str_m = '0' + str( m )
    else :
        str_m = str( m )
    str_d = ''
    if d < 10 :
        str_d = '0' + str( d )
    else :
        str_d = str( d )
    
    return str_y + str_m + str_d

#2. Label Encoding function
* 현재, train/ test dataset과 tube dataset과 merge하는 case까지 반영하여 encoder 관리

In [7]:
from sklearn import preprocessing

# le_bracket_pricing = preprocessing.LabelEncoder()
le_supplier = preprocessing.LabelEncoder()
le_material_id = preprocessing.LabelEncoder()
le_date = preprocessing.LabelEncoder()
le_year_month = preprocessing.LabelEncoder()

le_yn = preprocessing.LabelEncoder()
le_yn.fit( [ 'Y', 'N' ] )


le_end = preprocessing.LabelEncoder()
df_end = pd.read_csv( './dataset/01.original.dataset/tube_end_form.csv' )
le_end.fit( df_end[ 'end_form_id' ] )
df_end = ''


le_component_id = preprocessing.LabelEncoder()    # component의 경우, master dataset이 별도로 존재하므로
df_components = pd.read_csv( './dataset/02.ml.verified.dataset/components.verified.csv' )
le_component_id.fit( df_components[ 'component_id' ] )


le_spec_id = preprocessing.LabelEncoder()    # spec의 경우, spec meta dataset을 별도로 생성했음
df_spec_meta = pd.read_csv( './dataset/01.original.dataset/spec_meta.csv' )
le_spec_id.fit( df_spec_meta[ 'spec' ] )
df_spec_meta = ''


le_component_mac475 = preprocessing.LabelEncoder()
le_component_mac475.fit( df_components[ 'component_mac475' ] )


le_component_type_id = preprocessing.LabelEncoder()
le_component_type_id.fit( df_components[ 'component_type_id' ] )
df_components = ''


le_type_end = preprocessing.LabelEncoder()
df_type_end = pd.read_csv( './dataset/01.original.dataset/type_end_form.csv' )
le_type_end.fit( df_type_end[ 'end_form_id' ] )
df_type_end = ''


le_connection = preprocessing.LabelEncoder()
df_connection = pd.read_csv( './dataset/01.original.dataset/type_connection.csv' )
le_connection.fit( df_connection[ 'connection_type_id' ] )
df_connection = ''


le_yn_none = preprocessing.LabelEncoder()
le_yn_none.fit( [ 'Y', 'N', 'NONE' ] )


le_boss_type = preprocessing.LabelEncoder()
le_boss_type.fit( [ 'Boss', 'Stud', 'NONE' ] )


le_elbow_mj_class_code = preprocessing.LabelEncoder()
le_elbow_mj_class_code.fit( [ 'N', 'MJ-001', 'MJ-003', 'MJ-005', 'MJ-006', 'NONE' ] )

le_boss_outside_shape = preprocessing.LabelEncoder()
le_boss_outside_shape.fit( [ 'Round', 'Hex', 'NONE' ] )


le_boss_base_type = preprocessing.LabelEncoder()
le_boss_base_type.fit( [ 'Flat Bottom', 'Saddle', 'Shoulder', 'NONE' ] )

def executeLabelEncoding( p_df, is_init ) :
    if is_init == True :    # training dataset인 경우, label encoder 생성 및 fitting 수행
        p_df[ 'supplier' ] = le_supplier.fit_transform( p_df[ 'supplier' ] )
        p_df[ 'material_id' ] = le_material_id.fit_transform( p_df[ 'material_id' ] )
        p_df[ 'date' ] = le_date.fit_transform( p_df[ 'date' ] )
        p_df[ 'year_month' ] = le_year_month.fit_transform( p_df[ 'year_month' ] )
    else :
        p_df[ 'supplier' ] = le_supplier.transform( p_df[ 'supplier' ] )
        p_df[ 'material_id' ] = le_material_id.transform( p_df[ 'material_id' ] )
        p_df[ 'date' ] = le_date.transform( p_df[ 'date' ] )
        p_df[ 'year_month' ] = le_year_month.transform( p_df[ 'year_month' ] )
        
    # 무조건 수행
    p_df[ 'is_annual' ] = le_yn.transform( p_df[ 'is_annual' ] )
    p_df[ 'bracket_pricing' ] = le_yn.transform( p_df[ 'bracket_pricing' ] )
    
    # tube/material_id == null을 tube만 사용예측시 적용, bill/ spec 같이 사용예측시 주석처리
#     p_df[ 'end_a_1x' ] = le_yn.transform( p_df[ 'end_a_1x' ] )
#     p_df[ 'end_a_2x' ] = le_yn.transform( p_df[ 'end_a_2x' ] )
#     p_df[ 'end_x_1x' ] = le_yn.transform( p_df[ 'end_x_1x' ] )
#     p_df[ 'end_x_2x' ] = le_yn.transform( p_df[ 'end_x_2x' ] )
    
    p_df[ 'end_a' ] = le_end.transform( p_df[ 'end_a' ] )
    p_df[ 'end_x' ] = le_end.transform( p_df[ 'end_x' ] )

    for i in range( 1, 9 ) :    # bill_of_materials에서 merge된 component_id_1~8을 label encoding 수행
        comp_str = 'component_id_' + str( i )
        p_df[ comp_str ] = le_component_id.transform( p_df[ comp_str ] )

    for i in range( 1, 11 ) :    # specs에서 merge된 spec1~10을 label encoding 수행
        spec_str = 'spec' + str( i )
        p_df[ spec_str ] = le_spec_id.transform( p_df[ spec_str ] )                
        
    p_df[ 'forming_x' ] = le_yn.transform( p_df[ 'forming_x' ] )
    p_df[ 'forming_y' ] = le_yn.transform( p_df[ 'forming_y' ] )
    
    
    comp_list = [ 'x' ]
    for v in comp_list :
        v = ''
        p_df[ 'component_type_id' + v ] = le_component_type_id.transform( p_df[ 'component_type_id' + v ] )
        p_df[ 'component_mac475' + v ] = le_component_mac475.transform( p_df[ 'component_mac475' + v ] )


        p_df[ 'adaptor_end_form_id_1' + v ] = le_type_end.transform( p_df[ 'adaptor_end_form_id_1' + v ] )
        p_df[ 'adaptor_connection_type_id_1' + v ] = le_connection.transform( p_df[ 'adaptor_connection_type_id_1' + v ] )
        p_df[ 'adaptor_end_form_id_2' + v ] = le_type_end.transform( p_df[ 'adaptor_end_form_id_2' + v ] )
        p_df[ 'adaptor_connection_type_id_2' + v ] = le_connection.transform( p_df[ 'adaptor_connection_type_id_2' + v ] )
        p_df[ 'adaptor_unique_feature' + v ] = le_yn_none.transform( p_df[ 'adaptor_unique_feature' + v ] )
        p_df[ 'adaptor_orientation' + v ] = le_yn_none.transform( p_df[ 'adaptor_orientation' + v ] )


        p_df[ 'boss_type' + v ] = le_boss_type.transform( p_df[ 'boss_type' + v ] )
        p_df[ 'boss_connection_type_id' + v ] = le_connection.transform( p_df[ 'boss_connection_type_id' + v ] )
        p_df[ 'boss_outside_shape' + v ] = le_boss_outside_shape.transform( p_df[ 'boss_outside_shape' + v ] )
        p_df[ 'boss_base_type' + v ] = le_boss_base_type.transform( p_df[ 'boss_base_type' + v ] )
        p_df[ 'boss_groove' + v ] = le_yn_none.transform( p_df[ 'boss_groove' + v ] )
        p_df[ 'boss_unique_feature' + v ] = le_yn_none.transform( p_df[ 'boss_unique_feature' + v ] )
        p_df[ 'boss_orientation' + v ] = le_yn_none.transform( p_df[ 'boss_orientation' + v ] )


        p_df[ 'elbow_mj_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'elbow_mj_class_code' + v ] )
        p_df[ 'elbow_mj_plug_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'elbow_mj_plug_class_code' + v ] )
        p_df[ 'elbow_groove' + v ] = le_yn_none.transform( p_df[ 'elbow_groove' + v ] )
        p_df[ 'elbow_unique_feature' + v ] = le_yn_none.transform( p_df[ 'elbow_unique_feature' + v ] )
        p_df[ 'elbow_orientation' + v ] = le_yn_none.transform( p_df[ 'elbow_orientation' + v ] )

        p_df[ 'float_orientation' + v ] = le_yn_none.transform( p_df[ 'float_orientation' + v ] )

        p_df[ 'hfl_corresponding_shell' + v ] = le_yn_none.transform( p_df[ 'hfl_corresponding_shell' + v ] )
        p_df[ 'hfl_coupling_class' + v ] = le_yn_none.transform( p_df[ 'hfl_coupling_class' + v ] )
        p_df[ 'hfl_material' + v ] = le_yn_none.transform( p_df[ 'hfl_material' + v ] )
        p_df[ 'hfl_plating' + v ] = le_yn_none.transform( p_df[ 'hfl_plating' + v ] )
        p_df[ 'hfl_orientation' + v ] = le_yn_none.transform( p_df[ 'hfl_orientation' + v ] )

        p_df[ 'nut_blind_hole' + v ] = le_yn_none.transform( p_df[ 'nut_blind_hole' + v ] )
        p_df[ 'nut_orientation' + v ] = le_yn_none.transform( p_df[ 'nut_orientation' + v ] )

        p_df[ 'sleeve_connection_type_id' + v ] = le_connection.transform( p_df[ 'sleeve_connection_type_id' + v ] )    
        p_df[ 'sleeve_unique_feature' + v ] = le_yn_none.transform( p_df[ 'sleeve_unique_feature' + v ] )
        p_df[ 'sleeve_plating' + v ] = le_yn_none.transform( p_df[ 'sleeve_plating' + v ] )
        p_df[ 'sleeve_orientation' + v ] = le_yn_none.transform( p_df[ 'sleeve_orientation' + v ] )


        p_df[ 'straight_mj_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'straight_mj_class_code' + v ] )
        p_df[ 'straight_groove' + v ] = le_yn_none.transform( p_df[ 'straight_groove' + v ] )
        p_df[ 'straight_unique_feature' + v ] = le_yn_none.transform( p_df[ 'straight_unique_feature' + v ] )
        p_df[ 'straight_orientation' + v ] = le_yn_none.transform( p_df[ 'straight_orientation' + v ] )


        p_df[ 'tee_mj_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'tee_mj_class_code' + v ] )
        p_df[ 'tee_mj_plug_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'tee_mj_plug_class_code' + v ] )
        p_df[ 'tee_groove' + v ] = le_yn_none.transform( p_df[ 'tee_groove' + v ] )    
        p_df[ 'tee_unique_feature' + v ] = le_yn_none.transform( p_df[ 'tee_unique_feature' + v ] )    
        p_df[ 'tee_orientation' + v ] = le_yn_none.transform( p_df[ 'tee_orientation' + v ] )    

        p_df[ 'threaded_end_form_id_1' + v ] = le_type_end.transform( p_df[ 'threaded_end_form_id_1' + v ] )
        p_df[ 'threaded_connection_type_id_1' + v ] = le_connection.transform( p_df[ 'threaded_connection_type_id_1' + v ] )    
        p_df[ 'threaded_end_form_id_2' + v ] = le_type_end.transform( p_df[ 'threaded_end_form_id_2' + v ] )
        p_df[ 'threaded_connection_type_id_2' + v ] = le_connection.transform( p_df[ 'threaded_connection_type_id_2' + v ] )
        p_df[ 'threaded_end_form_id_3' + v ] = le_type_end.transform( p_df[ 'threaded_end_form_id_3' + v ] )
        p_df[ 'threaded_connection_type_id_3' + v ] = le_connection.transform( p_df[ 'threaded_connection_type_id_3' + v ] )
        p_df[ 'threaded_end_form_id_4' + v ] = le_end.transform( p_df[ 'threaded_end_form_id_4' + v ] )
        p_df[ 'threaded_connection_type_id_4' + v ] = le_connection.transform( p_df[ 'threaded_connection_type_id_4' + v ] )
        p_df[ 'threaded_unique_feature' + v ] = le_yn_none.transform( p_df[ 'threaded_unique_feature' + v ] )
        p_df[ 'threaded_orientation' + v ] = le_yn_none.transform( p_df[ 'threaded_orientation' + v ] )

    return p_df

.

#3. Label Encoding용 Dataset
* categorical data를 모두 포함해야 하므로, tube join후 기반으로 label encoder를 fitting한다
* label encoding의 대상은 bracket_pricing, supplier, material_id, end_a_1x 시리즈, end_a 시리즈 feature이다

##3.1 train/ test dataset merge

In [8]:
df_train[ 'id' ] = 99999    # test와 join위해 feature 추가 : 99999는 train dataset이다

In [9]:
df_merged = df_train.append( df_test )    # train과 test df를 merge한다.

##3.1 tube_bill_specs dataset merge 및 label encoding

In [10]:
df_tube_bill_specs_end_comp = pd.read_csv( './dataset/03.merged/tube_bill_specs_end_comp_merged.csv' )
# df_tube_bill_specs_end_comp = pd.read_csv( './dataset/03.merged/tube_bill_specs_end_merged.csv' )

# 임시코드
df_tube_bill_specs_end_comp.drop( [ 'length_1','length_2','length_3','length_4',
                                    'length_5','length_6','length_7','length_8',
#                                     'length_sum'
                                  ], axis = 1, inplace = True )
# 임시코드

In [11]:
dic = { 'Yes' : 'Y', 'No' : 'N' }
df_merged[ 'bracket_pricing' ].replace( dic, inplace = True )

In [12]:
def get_is_annual( p_df ) :    # annual_usage 존재여부를 확인
    is_annual = 'N'
    if p_df[ 'annual_usage' ] > 0 :
        is_annual = 'Y'
    return is_annual

In [13]:
df_merged[ 'is_annual' ] = df_merged.apply( get_is_annual, axis = 1 )

In [14]:
# min_order_quantity 조건테스트
df_merged[ 'min_order_quantity' ].replace( to_replace = 0, value = 1, inplace = True )

In [15]:
def get_order_count( p_df, p_type ) :    # 주문횟수를 계산
    order_count = 0
    
    if p_type == 'min' :
        if ( p_df[ 'annual_usage' ] > 0 ) & ( p_df[ 'min_order_quantity' ] > 0 ) :
            order_count = float( p_df[ 'annual_usage' ] / p_df[ 'min_order_quantity' ] )
    elif p_type == 'quantity' :
        order_count = float( p_df[ 'annual_usage' ] / p_df[ 'quantity' ] )
    elif p_type == 'rule' :
        if p_df[ 'bracket_pricing' ] == 'Y' :
            order_count = float( p_df[ 'annual_usage' ] / p_df[ 'quantity' ] )
        elif p_df[ 'bracket_pricing' ] == 'N' :
            order_count = float( p_df[ 'annual_usage' ] / p_df[ 'min_order_quantity' ] )
    elif p_type == 'sales' :
        order_count = float( p_df[ 'annual_usage' ] * p_df[ 'quantity' ] )
            
    return order_count

In [16]:
%%time
df_merged[ 'sales_result' ] = df_merged.apply( get_order_count, axis = 1, args = ('sales',) )
df_merged = extract_year_month_day_from_quote_date( df_merged )    # feature 처리를 수행

Wall time: 883 ms


In [17]:
%%time
df_merged[ 'date' ] = df_merged.apply( make_string_date, axis = 1 )
df_merged[ 'year_month' ] = df_merged[ 'date' ].str[0:6]

Wall time: 4.85 s


In [18]:
df_merged = df_merged.merge( df_tube_bill_specs_end_comp, how = 'inner', on = 'tube_assembly_id' )

In [19]:
%%time
df_merged = executeLabelEncoding( df_merged, True )

Wall time: 1.66 s


In [20]:
# weight_sum, length_sum의 nomalization을 위한 train/ test merged df에서의 weight_sum_max, length_sum_max
weight_sum_max = df_merged[ 'weight_sum' ].max()
length_sum_max = df_merged[ 'length_sum' ].max()

In [21]:
# df_merged.to_csv( './dataset/df_train_test_tube_bill_specs_end_comp_merged_encoded.csv', index = False )

In [22]:
df_train.drop( 'id', axis = 1, inplace = True )    # 일단, 필요없는 것들 제거 : df_train을 원상 복구해둠
del( df_test )    # 일단, 필요없는 것들 제거
del( df_merged )    # 일단, 필요없는 것들 제거

#4. Modeling

##4.1 Merged features중, modeling시 제거할 feature 관리

In [23]:
list_for_remove = []    # 전역변수의 선언
def executeFeatureRemoval( p_df ) :
    # 제거하고자 하는 feature list
    global list_for_remove    # 전역변수 명시
    
    list_for_remove = [
                        'tube_assembly_id',
                        'quote_date',
#                         'date',

#                         'supplier',        
#                         'annual_usage',
#                         'min_order_quantity',
#                         'bracket_pricing',
        
#                         'order_count_by_quantity',    # Case 88에서 제거시도
#                         'order_count_by_min',    # Case 88에서 제거시도
#                         'order_count_by_rule',    # Case 88에서 제거시도
#                         'sales_result',
        
        
#                         'quantity',
#                         'material_id',
#                         'diameter',
#                         'wall',
#                         'length',
#                         'num_bends',
#                         'bend_radius',
#                         'end_a_1x',
#                         'end_a_2x',
#                         'end_x_1x',
#                         'end_x_2x',
#                         'end_a',
#                         'end_x',
#                         'num_boss',
#                         'num_bracket',
#                         'other',
        
#                         'component_id_1',
#                         'quantity_1',
#                         'component_id_2',
#                         'quantity_2',
#                         'component_id_3',
#                         'quantity_3',
#                         'component_id_4',
#                         'quantity_4',

#                         'component_id_5',
#                         'quantity_5',
#                         'component_id_6',
#                         'quantity_6',
#                         'component_id_7',
#                         'quantity_7',
#                         'component_id_8',
#                         'quantity_8',
        
#                         'weight_sum',
#                         'adaptor_sum',
#                         'boss_sum',
#                         'elbow_sum',
#                         'float_sum',
#                         'hfl_sum',
#                         'nut_sum',
#                         'other_sum',
#                         'sleeve_sum',
#                         'straight_sum',
#                         'tee_sum',
#                         'threaded_sum',
        
#                         'adaptor_weight_sum',
#                         'boss_weight_sum',
#                         'elbow_weight_sum',
#                         'float_weight_sum',
#                         'hfl_weight_sum',
#                         'nut_weight_sum',
#                         'other_weight_sum',
#                         'sleeve_weight_sum',
#                         'straight_weight_sum',
#                         'tee_weight_sum',
#                         'threaded_weight_sum',
        
#                         'comp_type_count',
#                         'comp_total_count',
#                         'tube_volume',
        
#                         'spec1',
#                         'spec2',
#                         'spec3',
#                         'spec4',
#                         'spec5',
        
#                         'spec6',
#                         'spec7',
#                         'spec8',
#                         'spec9',
#                         'spec10',
        
#                         'spec_type_count',
#                         'forming_x',
#                         'forming_y',

                        # component family 계열의 merge를 하지않은 case에 대한 feature들을 일단 죽여둔다
                        'component_type_id',
                        'mac475_weight',
        
#                         'component_mac475',    # Case 87
                        
                        'uniqueness',
                        'orientation',
        
                        'adaptor_adaptor_angle', 'adaptor_overall_length', 'adaptor_end_form_id_1' ,
                        'adaptor_connection_type_id_1', 'adaptor_length_1', 'adaptor_thread_size_1',
                        'adaptor_thread_pitch_1','adaptor_nominal_size_1', 'adaptor_end_form_id_2',
                        'adaptor_connection_type_id_2','adaptor_length_2', 'adaptor_thread_size_2',
                        'adaptor_thread_pitch_2', 'adaptor_nominal_size_2', 'adaptor_hex_size',
                        'adaptor_unique_feature', 'adaptor_orientation', 'adaptor_weight',

                        'boss_type', 'boss_connection_type_id', 'boss_outside_shape', 'boss_base_type',
                        'boss_height_over_tube', 'boss_bolt_pattern_long', 'boss_bolt_pattern_wide',
                        'boss_groove', 'boss_base_diameter', 'boss_shoulder_diameter', 'boss_unique_feature',
                        'boss_orientation', 'boss_weight',

                        'elbow_bolt_pattern_long', 'elbow_bolt_pattern_wide', 'elbow_extension_length',
                        'elbow_overall_length', 'elbow_thickness', 'elbow_drop_length', 'elbow_elbow_angle',
                        'elbow_mj_class_code', 'elbow_mj_plug_class_code', 'elbow_plug_diameter',
                        'elbow_groove', 'elbow_unique_feature', 'elbow_orientation', 'elbow_weight',

                        'float_bolt_pattern_long', 'float_bolt_pattern_wide', 'float_thickness',
                        'float_orientation', 'float_weight',

                        'hfl_hose_diameter', 'hfl_corresponding_shell', 'hfl_coupling_class', 'hfl_material',
                        'hfl_plating', 'hfl_orientation', 'hfl_weight', 
        
                        'nut_hex_nut_size', 'nut_seat_angle', 'nut_length', 'nut_thread_size', 'nut_thread_pitch',
                        'nut_diameter', 'nut_blind_hole', 'nut_orientation', 'nut_weight',

                        'other_weight',

                        'sleeve_connection_type_id', 'sleeve_length', 'sleeve_intended_nut_thread',
                        'sleeve_intended_nut_pitch', 'sleeve_unique_feature', 'sleeve_plating', 'sleeve_orientation',
                        'sleeve_weight',

                        'straight_bolt_pattern_long', 'straight_bolt_pattern_wide', 'straight_head_diameter',
                        'straight_overall_length', 'straight_thickness', 'straight_mj_class_code',
                        'straight_groove', 'straight_unique_feature', 'straight_orientation', 'straight_weight',

                        'tee_bolt_pattern_long', 'tee_bolt_pattern_wide', 'tee_extension_length',
                        'tee_overall_length', 'tee_thickness', 'tee_drop_length', 'tee_mj_class_code',
                        'tee_mj_plug_class_code', 'tee_groove', 'tee_unique_feature', 'tee_orientation', 'tee_weight',

                        'threaded_adaptor_angle', 'threaded_overall_length', 'threaded_hex_size', 'threaded_end_form_id_1',
                        'threaded_connection_type_id_1', 'threaded_length_1', 'threaded_thread_size_1',
                        'threaded_thread_pitch_1', 'threaded_nominal_size_1', 'threaded_end_form_id_2',
                        'threaded_connection_type_id_2', 'threaded_length_2', 'threaded_thread_size_2',
                        'threaded_thread_pitch_2', 'threaded_nominal_size_2', 'threaded_end_form_id_3',
                        'threaded_connection_type_id_3', 'threaded_length_3', 'threaded_thread_size_3',
                        'threaded_thread_pitch_3', 'threaded_nominal_size_3', 'threaded_end_form_id_4',
                        'threaded_connection_type_id_4', 'threaded_length_4', 'threaded_thread_size_4',
                        'threaded_thread_pitch_4', 'threaded_nominal_size_4', 'threaded_unique_feature',
                        'threaded_orientation', 'threaded_weight',

                        # 예측결과 0.0000 영향도인 features
        
                        # 유의미 추정가능한 부활가능 Feature들 시작        
                        'num_bracket',
        
                        'component_id_4', 'component_id_5', 'component_id_6',        
                        'spec4','spec6','spec7','spec8',
                        'quantity_2','quantity_3','quantity_4','quantity_5','quantity_6','quantity_7',
                        'sleeve_weight_sum','hfl_weight_sum','float_weight_sum','boss_weight_sum','adaptor_weight_sum',
                        'tee_sum','straight_sum','sleeve_sum','hfl_sum','boss_sum','adaptor_sum','elbow_sum',
                        'comp_total_count',
        
#                         'bracket_pricing',        
                        'forming_y','forming_x',
                        'spec_type_count','comp_type_count',
                        # 유의미 추정가능한 부활가능 Feature들 종료

                        'is_annual',
                        'component_id_7','component_id_8',
                        'spec9','spec10',
                        'uniqueness_count',
                        'quantity_8',
                        'tee_weight_sum',
                        'SP-0060', 'SP-0063', 'SP-0062', 'SP-0064', 'SP-0070', 'SP-0065', 'SP-0066', 'SP-0067', 'SP-0068',
                        'SP-0051', 'SP-0059', 'SP-0056', 'SP-0055','SP-0054','SP-0053','SP-0052','SP-0072','SP-0050',
                        'SP-0049','SP-0048',  'SP-0046','SP-0045', 'SP-0071','SP-0081','SP-0073','SP-0074',
                        'SP-0096','SP-0095','SP-0094','SP-0093','SP-0092','SP-0091','SP-0090','SP-0089','SP-0088',
                        'SP-0087','SP-0086','SP-0085','SP-0084','SP-0083','SP-0082','SP-0043','SP-0080','SP-0079','SP-0078',
                        'SP-0077','SP-0076','SP-0075','SP-0044','SP-0038','SP-0042','SP-0001','SP-0002','SP-0003',
                        'SP-0004','SP-0006','SP-0041','SP-0033','SP-0027','SP-0028','SP-0029','SP-0030','SP-0031',
                        'SP-0032','SP-0034','SP-0007','SP-0035','SP-0036','SP-0037','SP-0039','SP-0040','SP-0026',
                        'SP-0025','SP-0024','SP-0023','SP-0021','SP-0020','SP-0019','SP-0018','SP-0015','SP-0014',
                        'SP-0013','SP-0012','SP-0011','SP-0010','SP-0009','SP-0008','SP-0017',
                        'price_band_count',
                        'bracket_yes_count', 'bracket_no_count',
                        'tube_area',
                        'overall_length',
#                         'complexity',
#                         'length_sum',
                        'year_month',
                      ]

    return p_df.drop( list_for_remove, axis = 1, inplace = False )

##4.2 Train : 연/ 월/ 일 Feature 추출

In [24]:
%%time
df_train = extract_year_month_day_from_quote_date( df_train )    # feature 처리를 수행
df_train[ 'date' ] = df_train.apply( make_string_date, axis = 1 )
df_train[ 'year_month' ] = df_train[ 'date' ].str[0:6]

Wall time: 2.38 s


##4.3 Train : Tube와 merge통한 Feature 확장

In [25]:
df_train_merged = df_train.merge( df_tube_bill_specs_end_comp, how = 'inner', on = 'tube_assembly_id' )

##Train : Tube별 min, max quantity Feature 확장

In [26]:
df_tube_min_max_quantity = pd.read_csv( './dataset/01.original.dataset/tube.min.max.quantity.csv' )

In [27]:
df_train_merged = df_train_merged.merge( df_tube_min_max_quantity, how = 'inner', on = 'tube_assembly_id' )

##Train : supplier annual 확장

In [28]:
# df_supplier_annual = pd.read_csv( './dataset/01.original.dataset/supplier_annual.csv' )
# df_supplier_total = pd.read_csv( './dataset/01.original.dataset/supplier_total.csv' )

In [29]:
# df_train_merged = df_train_merged.merge( df_supplier_annual, how = 'inner', on = [ 'supplier', 'year' ] )
# df_train_merged = df_train_merged.merge( df_supplier_total, how = 'inner', on = [ 'supplier' ] )

##4.4 Train : Merged dataset Label Encoding 수행

In [30]:
df_train_merged[ 'bracket_pricing' ].replace( dic, inplace = True )

In [31]:
%%time
df_train_merged[ 'is_annual' ] = df_train_merged.apply( get_is_annual, axis = 1 )
df_train_merged[ 'order_count_by_min' ] = df_train_merged.apply( get_order_count, axis = 1, args = ('min',) )
df_train_merged[ 'order_count_by_quantity' ] = df_train_merged.apply( get_order_count, axis = 1, args = ('quantity',) )
df_train_merged[ 'order_count_by_rule' ] = df_train_merged.apply( get_order_count, axis = 1, args = ('rule',) )
df_train_merged[ 'sales_result' ] = df_train_merged.apply( get_order_count, axis = 1, args = ('sales',) )

Wall time: 11.9 s


In [32]:
df_train_merged[ 'req_comp_spec_type_count' ] = df_train_merged[ 'spec_type_count' ] + df_train_merged[ 'comp_type_count' ]
df_train_merged[ 'req_comp_spec_total_count' ] = df_train_merged[ 'spec_type_count' ] + df_train_merged[ 'comp_type_count' ] * df_train_merged[ 'comp_total_count' ]

In [33]:
def calculate_end_ax_count( p_df ) :    # end_a, end_x의 count를 계산
    count = 0
    if p_df[ 'end_a' ] != 'NONE' :
        count += 1
    if p_df[ 'end_x' ] != 'NONE' :
        count += 1
    return count

def check_is_end_ax_same( p_df ) :    # end_a, end_x가 동일한지 여부를 판단
    is_same = 0
    if p_df[ 'end_a' ] == p_df[ 'end_x' ] :
        is_same = 1
    return is_same

In [34]:
# df_train_merged[ 'end_ax_count' ] = df_train_merged.apply( calculate_end_ax_count, axis = 1 )

In [35]:
# df_train_merged[ 'is_end_ax_same' ] = df_train_merged.apply( check_is_end_ax_same, axis = 1 )

##4.5 Train : 불필요 Feature 제거

In [36]:
# df_train_merged.to_csv( './dataset/df_train_tube_bill_specs_end_comp_merged.csv' )
# df_train_merged = executeLabelEncoding( df_train_merged, is_init = False )    # label encoding을 수행한다
# df_train_merged.to_csv( './dataset/df_train_tube_bill_specs_end_comp_merged_encoded.csv' )
# df_train_merged = executeFeatureRemoval( df_train_merged )    # feature removal 처리를 수행

.

##4.6 Test : Dataset 읽기

In [37]:
df_test = pd.read_csv( './dataset/test_set.csv' )    # data를 읽어들인다.

In [38]:
if bracket == 'Yes' :
    # bracket_pricing == Yes인 경우만
    df_test = df_test[ df_test[ 'bracket_pricing' ] == 'Yes' ]
elif bracket == 'No' :
    # bracket_pricing == No인 경우만
    df_test = df_test[ df_test[ 'bracket_pricing' ] == 'No' ]

##4.7 Test : 연/ 월/ 일 Feature 추출

In [39]:
%%time
df_test = extract_year_month_day_from_quote_date( df_test )    # feature 처리를 수행
df_test[ 'date' ] = df_test.apply( make_string_date, axis = 1 )
df_test[ 'year_month' ] = df_test[ 'date' ].str[0:6]

df_result = pd.DataFrame( df_test[ 'id' ], columns = ['id'] )    # 결과용 dataframe을 생성
df_test.drop( 'id', axis = 1, inplace = True )    # id feature는 일단 제거

Wall time: 2.37 s


##4.8 Test : Tube와 merge통한 Feature 확장

In [40]:
df_test_merged = df_test.merge( df_tube_bill_specs_end_comp, how = 'inner', on = 'tube_assembly_id' )

In [41]:
df_test_merged = df_test_merged.merge( df_tube_min_max_quantity, how = 'inner', on = 'tube_assembly_id' )

In [42]:
# df_test_merged = df_test_merged.merge( df_supplier_annual, how = 'inner', on = [ 'supplier', 'year' ] )
# df_test_merged = df_test_merged.merge( df_supplier_total, how = 'inner', on = [ 'supplier' ] )
# del( df_supplier_annual )
# del( df_supplier_total )

##4.9 Test : Merged dataset Label Encoding 수행

In [43]:
%%time
df_test_merged[ 'bracket_pricing' ].replace( dic, inplace = True )
df_test_merged[ 'is_annual' ] = df_test_merged.apply( get_is_annual, axis = 1 )
df_test_merged[ 'order_count_by_min' ] = df_test_merged.apply( get_order_count, axis = 1, args = ('min',) )
df_test_merged[ 'order_count_by_quantity' ] = df_test_merged.apply( get_order_count, axis = 1, args = ('quantity',) )
df_test_merged[ 'order_count_by_rule' ] = df_test_merged.apply( get_order_count, axis = 1, args = ('rule',) )
df_test_merged[ 'sales_result' ] = df_test_merged.apply( get_order_count, axis = 1, args = ('sales',) )

df_test_merged[ 'req_comp_spec_type_count' ] = df_test_merged[ 'spec_type_count' ] + df_test_merged[ 'comp_type_count' ]
df_test_merged[ 'req_comp_spec_total_count' ] = df_test_merged[ 'spec_type_count' ] + df_test_merged[ 'comp_type_count' ] * df_test_merged[ 'comp_total_count' ]

# df_test_merged[ 'end_ax_count' ] = df_test_merged.apply( calculate_end_ax_count, axis = 1 )
# df_test_merged[ 'is_end_ax_same' ] = df_test_merged.apply( check_is_end_ax_same, axis = 1 )

Wall time: 12 s


##4.10 Test : 불필요 Featrue 제거

In [44]:
%%time
# bend num으로 인한 tube의 복잡도 추정 feature의 추가
replaced_val = 1
df_train_merged[ 'bend_num_by_radius' ] = df_train_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_train_merged[ 'bend_radius' ]
df_test_merged[ 'bend_num_by_radius' ] = df_test_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_test_merged[ 'bend_radius' ]

df_train_merged[ 'bend_num_X_volume' ] = df_train_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_train_merged[ 'tube_volume' ]
df_test_merged[ 'bend_num_X_volume' ] = df_test_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_test_merged[ 'tube_volume' ]

df_train_merged[ 'bend_num_X_length' ] = df_train_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_train_merged[ 'length' ]
df_test_merged[ 'bend_num_X_length' ] = df_test_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_test_merged[ 'length' ]

df_train_merged[ 'bend_num_X_diameter' ] = df_train_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_train_merged[ 'diameter' ]
df_test_merged[ 'bend_num_X_diameter' ] = df_test_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_test_merged[ 'diameter' ]

df_train_merged[ 'bend_num_X_wall' ] = df_train_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_train_merged[ 'wall' ]
df_test_merged[ 'bend_num_X_wall' ] = df_test_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_test_merged[ 'wall' ]


# df_train_merged[ 'bend_num_X_area' ] = df_train_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_train_merged[ 'tube_area' ]
# df_test_merged[ 'bend_num_X_area' ] = df_test_merged[ 'num_bends' ].replace( 0, replaced_val ) * df_test_merged[ 'tube_area' ]

df_train_merged[ 'quantity_by_max' ] = df_train_merged[ 'quantity' ] /  df_train_merged[ 'quantity_range_max' ]
df_test_merged[ 'quantity_by_max' ] = df_test_merged[ 'quantity' ] /  df_test_merged[ 'quantity_range_max' ]
df_train_merged[ 'quantity_by_min' ] = df_train_merged[ 'quantity' ] /  df_train_merged[ 'quantity_range_min' ]
df_test_merged[ 'quantity_by_min' ] = df_test_merged[ 'quantity' ] /  df_test_merged[ 'quantity_range_min' ]

# 추가
df_train_merged[ 'bracket_num_X_volume' ] = df_train_merged[ 'num_bracket' ] * df_train_merged[ 'tube_volume' ]
df_test_merged[ 'bracket_num_X_volume' ] = df_test_merged[ 'num_bracket' ] * df_test_merged[ 'tube_volume' ]
df_train_merged[ 'bracket_num_X_wall' ] = df_train_merged[ 'num_bracket' ] * df_train_merged[ 'wall' ]
df_test_merged[ 'bracket_num_X_wall' ] = df_test_merged[ 'num_bracket' ] * df_test_merged[ 'wall' ]

# 복잡도들의 합
df_train_merged[ 'complexity' ] = df_train_merged[ 'num_bracket' ] +\
                                  df_train_merged[ 'num_boss' ] + df_train_merged[ 'other' ]
df_test_merged[ 'complexity' ] = df_test_merged[ 'num_bracket' ] +\
                                 df_test_merged[ 'num_boss' ] + df_test_merged[ 'other' ]
# df_train_merged[ 'complexity' ] = df_train_merged[ 'num_bends' ] + df_train_merged[ 'num_bracket' ] +\
#                                   df_train_merged[ 'num_boss' ] + df_train_merged[ 'other' ]
# df_test_merged[ 'complexity' ] = df_test_merged[ 'num_bends' ] + df_test_merged[ 'num_bracket' ] +\
#                                  df_test_merged[ 'num_boss' ] + df_test_merged[ 'other' ]
df_train_merged[ 'complexity_X_volume' ] = df_train_merged[ 'complexity' ] * df_train_merged[ 'tube_volume' ]
df_test_merged[ 'complexity_X_volume' ] = df_test_merged[ 'complexity' ] * df_test_merged[ 'tube_volume' ]
df_train_merged[ 'complexity_X_diameter' ] = df_train_merged[ 'complexity' ] * df_train_merged[ 'diameter' ]
df_test_merged[ 'complexity_X_diameter' ] = df_test_merged[ 'complexity' ] * df_test_merged[ 'diameter' ]


# weight_sum, length_sum의 활용
# df_train_merged[ 'weight_sum' ] = df_train_merged[ 'weight_sum' ] / weight_sum_max
# df_test_merged[ 'weight_sum' ] = df_test_merged[ 'weight_sum' ] / weight_sum_max
# df_train_merged[ 'length_sum' ] = df_train_merged[ 'length_sum' ] / length_sum_max
# df_test_merged[ 'length_sum' ] = df_test_merged[ 'length_sum' ] / length_sum_max
# df_train_merged[ 'weight_sum_plus_length_sum' ] = df_train_merged[ 'weight_sum' ] + df_train_merged[ 'length_sum' ]
# df_test_merged[ 'weight_sum_plus_length_sum' ] = df_test_merged[ 'weight_sum' ] + df_test_merged[ 'length_sum' ] 

# #### weight_sum, length_sum, complexity
# df_train_merged[ 'weight_sum_X_volume' ] = df_train_merged[ 'weight_sum' ].replace( 0, replaced_val ) * df_train_merged[ 'tube_volume' ]
# df_test_merged[ 'weight_sum_X_volume' ] = df_test_merged[ 'weight_sum' ].replace( 0, replaced_val ) * df_test_merged[ 'tube_volume' ]

# df_train_merged[ 'weight_sum_X_length' ] = df_train_merged[ 'weight_sum' ].replace( 0, replaced_val ) * df_train_merged[ 'length' ]
# df_test_merged[ 'weight_sum_X_length' ] = df_test_merged[ 'weight_sum' ].replace( 0, replaced_val ) * df_test_merged[ 'length' ]

# df_train_merged[ 'weight_sum_X_diameter' ] = df_train_merged[ 'weight_sum' ].replace( 0, replaced_val ) * df_train_merged[ 'diameter' ]
# df_test_merged[ 'weight_sum_X_diameter' ] = df_test_merged[ 'weight_sum' ].replace( 0, replaced_val ) * df_test_merged[ 'diameter' ]

# df_train_merged[ 'weight_sum_X_wall' ] = df_train_merged[ 'weight_sum' ].replace( 0, replaced_val ) * df_train_merged[ 'wall' ]
# df_test_merged[ 'weight_sum_X_wall' ] = df_test_merged[ 'weight_sum' ].replace( 0, replaced_val ) * df_test_merged[ 'wall' ]


# df_train_merged[ 'length_sum_X_volume' ] = df_train_merged[ 'length_sum' ].replace( 0, replaced_val ) * df_train_merged[ 'tube_volume' ]
# df_test_merged[ 'length_sum_X_volume' ] = df_test_merged[ 'length_sum' ].replace( 0, replaced_val ) * df_test_merged[ 'tube_volume' ]

# df_train_merged[ 'length_sum_X_length' ] = df_train_merged[ 'length_sum' ].replace( 0, replaced_val ) * df_train_merged[ 'length' ]
# df_test_merged[ 'length_sum_X_length' ] = df_test_merged[ 'length_sum' ].replace( 0, replaced_val ) * df_test_merged[ 'length' ]

# df_train_merged[ 'length_sum_X_diameter' ] = df_train_merged[ 'length_sum' ].replace( 0, replaced_val ) * df_train_merged[ 'diameter' ]
# df_test_merged[ 'length_sum_X_diameter' ] = df_test_merged[ 'length_sum' ].replace( 0, replaced_val ) * df_test_merged[ 'diameter' ]

# df_train_merged[ 'length_sum_X_wall' ] = df_train_merged[ 'length_sum' ].replace( 0, replaced_val ) * df_train_merged[ 'wall' ]
# df_test_merged[ 'length_sum_X_wall' ] = df_test_merged[ 'length_sum' ].replace( 0, replaced_val ) * df_test_merged[ 'wall' ]


# df_train_merged[ 'complexity_X_volume' ] = df_train_merged[ 'complexity' ].replace( 0, replaced_val ) * df_train_merged[ 'tube_volume' ]
# df_test_merged[ 'complexity_X_volume' ] = df_test_merged[ 'complexity' ].replace( 0, replaced_val ) * df_test_merged[ 'tube_volume' ]

# df_train_merged[ 'complexity_X_length' ] = df_train_merged[ 'complexity' ].replace( 0, replaced_val ) * df_train_merged[ 'length' ]
# df_test_merged[ 'complexity_X_length' ] = df_test_merged[ 'complexity' ].replace( 0, replaced_val ) * df_test_merged[ 'length' ]

# df_train_merged[ 'complexity_X_diameter' ] = df_train_merged[ 'complexity' ].replace( 0, replaced_val ) * df_train_merged[ 'diameter' ]
# df_test_merged[ 'complexity_X_diameter' ] = df_test_merged[ 'complexity' ].replace( 0, replaced_val ) * df_test_merged[ 'diameter' ]

# df_train_merged[ 'complexity_X_wall' ] = df_train_merged[ 'complexity' ].replace( 0, replaced_val ) * df_train_merged[ 'wall' ]
# df_test_merged[ 'complexity_X_wall' ] = df_test_merged[ 'complexity' ].replace( 0, replaced_val ) * df_test_merged[ 'wall' ]

Wall time: 15.6 ms


In [45]:
list_to_log = [ 'tube_volume',
                'bend_num_X_volume', 'bend_num_X_length', 'bend_num_X_diameter', 'bend_num_X_wall', 'bend_num_by_radius',
                'bracket_num_X_volume',
                'complexity_X_volume',
                'complexity_X_diameter',
              ]

for feat in list_to_log :
    df_train_merged[ feat ].replace( to_replace = 0, value = 1, inplace = True )
    df_test_merged[ feat ].replace( to_replace = 0, value = 1, inplace = True )
    
    df_train_merged[ feat ] = np.log10( df_train_merged[ feat ] )
    df_test_merged[ feat ] = np.log10( df_test_merged[ feat ] )

In [46]:
if bracket != 'All' :
    df_train_merged.drop( 'bracket_pricing', axis = 1, inplace = False )
    df_test_merged.drop( 'bracket_pricing', axis = 1, inplace = False )

In [47]:
%%time
df_train_merged.to_csv( './dataset/df_train_tube_bill_specs_end_comp_merged.csv' )
df_train_merged = executeLabelEncoding( df_train_merged, is_init = False )    # label encoding을 수행한다
df_train_merged.to_csv( './dataset/df_train_tube_bill_specs_end_comp_merged_encoded.csv' )
df_train_merged = executeFeatureRemoval( df_train_merged )    # feature removal 처리를 수행
df_train_merged.to_csv( './dataset/df_train_tube_bill_specs_end_comp_merged_encoded_final.csv' )

Wall time: 8.78 s


In [48]:
%%time
df_test_merged = executeLabelEncoding( df_test_merged, is_init = False )    # label encoding을 수행한다
df_test_merged = executeFeatureRemoval( df_test_merged )    # feature removal 처리를 수행

Wall time: 1.45 s


##4.11 Prediction Modeling

In [49]:
X = df_train_merged.drop( 'cost', axis = 1, inplace = False )    # X를 확보
y = np.log1p( df_train_merged[ 'cost' ] )   # y를 확보

del( df_train )
del( df_train_merged )

In [50]:
cv_cnt = 10    # cv : cross validation 횟수
n_jobs_cnt = 6

In [51]:
from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn import grid_search

In [ ]:
if mode == True :
    model_list = [
                   GradientBoostingRegressor(
                                              n_estimators = 2000,
                                              max_depth = 7,
                                              warm_start = True,
                                              subsample = 0.8
                                            ),
                 ]
else :
    model_list = [
#                    RandomForestRegressor(),    # 테스트용
                   GradientBoostingRegressor(),    # 테스트용
                 ]    

In [ ]:
%%time
import time
for model in model_list :
    for rep_counter in range( 0, predict_counter ) :    # rep_counter만큼 반복한다
        model_name = str( model ).split( '(' )[0]    # 파일생성용

        if model_name.startswith( 'GradientBoostingRegressor' ) :
            if mode == True : 
                params = {
    #                        'n_estimators' : (2500,3000),
    #                        'max_depth' : (5,7,9),
    #                        'subsample' : (1.0,0.9,0.8,0.7),
    #                        'loss' : ('ls','lad','huber'),
                         }
            else :
                params = {}
        elif model_name.startswith( 'RandomForestRegressor' ) :
            params = {
                     }        

        gs = grid_search.GridSearchCV( model,
                                       param_grid = params,
                                       n_jobs = n_jobs_cnt,
                                       cv = cv_cnt,
                                     )
        gs.fit( X, y )

        print( '=========================================' )
        print( 'model : ', str( model ).split( '(' )[0] )
        print( 'best_score : ', gs.best_score_ )

#         print( '=================' )
#         print( 'best model : ', gs.best_estimator_ )
#         print( '=================' )

        df_feature_importance = pd.DataFrame( X.columns.values, columns = [ 'features' ] )
        df_feature_importance[ 'importance' ] = gs.best_estimator_.feature_importances_ * 100
#         print( df_feature_importance.sort( 'importance', ascending = False  ) )    


        y_pred = gs.best_estimator_.predict( df_test_merged )    # prediction 수행
        df_result[ 'cost' ] = np.expm1( y_pred )

        now = time.strftime( '%Y%m%d%H%M%S' )    # 현재시각을 확보
        file_timestamp = now[2:4] + now[4:6] + now[6:8] + now[8:10] + now[10:12] + now[12:14]
        accuracy = '{0:.2f}%'.format( gs.best_score_ * 100 )    # latitude에 대한 예측률 저장 : 파일명 활용용도임

        if bracket == 'Yes' :
            df_result.to_csv( path_or_buf = './result.to.submit/' + file_timestamp + '.' + model_name +
                              '.' + accuracy + '.result().bracket.Y.csv', sep = ',', index = False )
        elif bracket == 'No' :
            df_result.to_csv( path_or_buf = './result.to.submit/' + file_timestamp + '.' + model_name +
                              '.' + accuracy + '.result().bracket.N.csv', sep = ',', index = False )
        elif bracket == 'All' :
            df_result.to_csv( path_or_buf = './result.to.submit/Case145s/' + file_timestamp + '.' + model_name +
                              '.' + accuracy + '.result().bracket.All.csv', sep = ',', index = False )                


        list_features = df_test_merged.columns.values.tolist()    

        with open( 'performance.condition.history.txt', 'a' ) as history_file :    # 모델의 이력을 logging
            history_file.write( '======================================================================\n' )
            history_file.write( '0. Description :\n' )
            if mode == True :
                history_file.write( '\tmode : Production mode\n' )
            else :
                history_file.write( '\tmode : Test mode\n' )
            history_file.write( '1. File name :\n\t' + file_timestamp + '.' + model_name + '.' + accuracy + '\n' )
            history_file.write( '\t' + str( gs.best_score_ ) + '\n' )
            history_file.write( '2. Model information :\n\t' + str( gs.best_estimator_ ) + '\n' )
            history_file.write( '3. Applied features :\n' )
            for feature in list_features :
                history_file.write( '\t' + feature + '\n' )
            history_file.write( '4. Removed features :\n' )
            for feature in list_for_remove :
                history_file.write( '\t' + feature + '\n' )
            history_file.write( '5. Feature importance :\n\t' )        
            history_file.write( df_feature_importance.sort( 'importance', ascending = False  ).to_string() )
            history_file.write( '\n' )
            history_file.close()
        rep_counter += 1

In [ ]:
df_result.head( 3 )

In [ ]:
df_result_desc = df_result.describe()    # 결과의 overview 확인
df_result_desc.drop( 'id', axis = 1, inplace = True )
df_result_desc.describe()

In [ ]:
del( X )
del( y )
del( df_test )
del( df_test_merged )
del( df_components )
del( df_result_desc )
del( df_result )
del( df_tube_min_max_quantity )

- 1개 0.5시간
- 5개 2.5시간
- 10개 5시간
- 20개 10시간
- 30개 15시간
- 40개 20시간 하루
- 50개 30시간
- 80개 40시간 
- 100개 50시간 이틀